In [1]:
from datetime import date
from tqdm import tqdm
import shutil
import os

from thumbnails import ThumbnailDef, Thumbnailer, ThumbnailParams
from db_connect import DbConnector
from photo_export import PhotoExporter
from path_management import PathManager
from bucket_connect import BucketConnector

In [2]:
from_date = date(2016, 10, 1)
to_date = date(2016, 11, 1)

In [3]:
small = ThumbnailDef(120, 'small')
medium = ThumbnailDef(400, 'medium')
large = ThumbnailDef(1600, 'large')

sizes = [small, large, medium] 
thumbnailer = Thumbnailer(sizes)

In [4]:
paths = PathManager(thumbnailer.name_modifiers)
test_connector = DbConnector('./test.postgres_config', paths)
prod_connector = DbConnector('./prod.postgres_config', paths)
exporter = PhotoExporter(paths)

games = prod_connector.get_games(from_date, to_date)

In [6]:
for game in tqdm(games):
    exporter.export_photos_for_game(game)

  0%|          | 0/6 [00:00<?, ?it/s]

skipping missing photo ADAC107C-2F8D-4D0F-8E2D-945A6745D210.JPG
skipping missing photo FD32F282-9878-4D9A-BB21-A8AD7DD0758D.JPG
skipping missing photo 7C38ED79-D0CB-4F79-A709-6379C862FCB4.JPG


 17%|█▋        | 1/6 [00:01<00:05,  1.14s/it]

skipping missing photo 54980F5A-D78A-4A9D-897C-89F1A9EEC54A.JPG


 33%|███▎      | 2/6 [00:01<00:03,  1.13it/s]

...

 67%|██████▋   | 4/6 [00:03<00:01,  1.12it/s]

skipping missing photo 3E8A8491-817B-4E16-B279-C7600BEE221F.JPG
skipping missing photo 5F759762-62FA-48C3-A603-46312671E558.JPG


 83%|████████▎ | 5/6 [00:04<00:00,  1.15it/s]

skipping missing photo 02DE7AA0-1476-4453-A439-0B3097FAE67D.JPG


100%|██████████| 6/6 [00:04<00:00,  1.26it/s]


At this point, review the photos in the "preview" directories and remove any that shouldn't be uploaded. Removing it from the preview directory is sufficient.

In [7]:
to_thumbnail: list[ThumbnailParams] = []
for game in games:
    to_thumbnail.extend(exporter.get_photos_to_thumbnail(game))

In [8]:
thumbnailer.thumbnail_many(to_thumbnail)

In [9]:
bucket = BucketConnector('./prod.bucket_config', paths)

In [10]:
for game in games:
    to_upload = exporter.get_photos_to_upload(game)
    print(f'Uploading/Validating {len(to_upload)} photos for {game.Name}')
    for photo in to_upload:
        root_path = paths.temp_dir(game, photo)
        bucket.upload_all_files(root_path, photo)
    prod_connector.import_resources(game, to_upload)

Uploading/Validating 35 photos for 10/15/16 NLCS Game 1: Los Angeles Dodgers at Chicago Cubs
Uploading/Validating 22 photos for 10/7/16 NLDS Game 1: San Francisco Giants at Chicago Cubs
Uploading/Validating 8 photos for 10/16/16 NLCS Game 2: Los Angeles Dodgers at Chicago Cubs
Uploading/Validating 41 photos for 10/28/16 WS Game 3: Cleveland Indians at Chicago Cubs
Uploading/Validating 22 photos for 10/1/16 Toronto Blue Jays at Boston Red Sox
Uploading/Validating 13 photos for 10/8/16 NLDS Game 2: San Francisco Giants at Chicago Cubs


In [11]:
for game in games:
    root_path = paths.base_dir(game)
    if os.path.exists(root_path):
        shutil.rmtree(root_path)